In [1]:


import findspark
findspark.init()



In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName("MultipleLinearRegresyon2") \
.master("local[4]") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory","4g") \
.getOrCreate()

# Veri Okuma

In [3]:
df = spark.read.format("csv") \
.option("header",True) \
.option("inferSchema",True) \
.option("sep",",") \
.load("/home/taha/Downloads/Life Expectancy Data.csv")

In [4]:
df.toPandas().head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


# Nitelik İSimlerini Degistirelim

In [5]:
new_cols = ["Country", "Year", "Status", "label", "AdultMortality",
      "InfantDeaths", "Alcohol", "PercentageExpenditure", "HepatitisB", "Measles", "BMI", "UnderFiveDeaths",
      "Polio", "TotalExpenditure", "Diphtheria", "HIV_AIDS", "GDP", "Population", "Thinness119", "Thinness59",
      "IncomeCompositionOfResources", "Schooling"]


In [6]:
df2 = df.toDF(*new_cols)

In [7]:
df2.toPandas().head()

,Country,Year,Status,label,AdultMortality,InfantDeaths,Alcohol,PercentageExpenditure,HepatitisB,Measles,...,Polio,TotalExpenditure,Diphtheria,HIV_AIDS,GDP,Population,Thinness119,Thinness59,IncomeCompositionOfResources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


# Kategorileri Ayiralim

In [226]:
"""
numerical_cols = [ "Year", "AdultMortality",
      "InfantDeaths", "Alcohol", "PercentageExpenditure", "HepatitisB", "Measles", "BMI", "UnderFiveDeaths",
      "Polio", "TotalExpenditure", "Diphtheria", "HIV_AIDS", "GDP", "Population", "Thinness119", "Thinness59",
      "IncomeCompositionOfResources", "Schooling"]
"""

numerical_cols = [ "Year", "AdultMortality",
      "InfantDeaths", "Alcohol",   "BMI", "UnderFiveDeaths",
       "TotalExpenditure", "Diphtheria", "HIV_AIDS", "GDP",
      "IncomeCompositionOfResources", "Schooling"]



categorical_cols = ["Country","Status"]

label = ["label"]

### Bunlari bu sekilde ayirmamizin sebebi regresyon modelinde
### cikarma yaparken islemleri kolaylastirmamiz

# Veri Temizligi

In [227]:
df2.describe().toPandas().head()

,summary,Country,Year,Status,label,AdultMortality,InfantDeaths,Alcohol,PercentageExpenditure,HepatitisB,...,Polio,TotalExpenditure,Diphtheria,HIV_AIDS,GDP,Population,Thinness119,Thinness59,IncomeCompositionOfResources,Schooling
0,count,2938,2938,2938,2928,2928,2938,2744,2938,2385,...,2919,2712,2919,2938,2490,2286,2904,2904,2771,2775
1,mean,None,2007.5187202178352,None,69.22493169398912,164.79644808743168,30.303948264125257,4.6028607871720375,738.2512954533823,80.94046121593291,...,82.55018842069202,5.9381895280235915,82.32408359027065,1.7421034717494939,7483.158469138481,1.2753375120052498E7,4.8397038567493205,4.870316804407711,0.6275510645976166,11.992792792792786
2,stddev,None,4.613840940258099,None,9.523867487824305,124.2920790034219,117.92650131339906,4.052412658755658,1987.914858016194,25.070015593018063,...,23.428045949468476,2.498319672155633,23.7169120685726,5.077784531086547,14270.169341515959,6.101209650842794E7,4.420194947144322,4.508882086983007,0.2109035551515931,3.3589197211023554
3,min,Afghanistan,2000,Developed,36.3,1,0,0.01,0.0,1,...,3,0.37,2,0.1,1.68135,34.0,0.1,0.1,0.0,0.0
4,max,Zimbabwe,2015,Developing,89.0,723,1800,17.87,19479.91161,99,...,99,17.6,99,50.6,119172.7418,1.293859294E9,27.7,28.6,0.948,20.7


In [228]:
df3 = df2.na.drop()

In [229]:
df3.describe().toPandas().head()

,summary,Country,Year,Status,label,AdultMortality,InfantDeaths,Alcohol,PercentageExpenditure,HepatitisB,...,Polio,TotalExpenditure,Diphtheria,HIV_AIDS,GDP,Population,Thinness119,Thinness59,IncomeCompositionOfResources,Schooling
0,count,1649,1649,1649,1649,1649,1649,1649,1649,1649,...,1649,1649,1649,1649,1649,1649,1649,1649,1649,1649
1,mean,None,2007.8405093996362,None,69.30230442692543,168.2152819890843,32.55306246209824,4.533195876288676,698.973558049698,79.21770770163735,...,83.56458459672528,5.955924802910845,84.155245603396,1.9838690115221154,5566.031886817592,1.4653625889484541E7,4.850636749545183,4.907762280169795,0.6315512431776829,12.119890842935117
2,stddev,None,4.087710534059055,None,8.796834135238601,125.31041693156172,120.84719049638885,4.029188999208425,1759.2293363477393,25.60466388103202,...,22.450557485191844,2.2993854200363937,21.57919284379662,6.032359689186714,11475.900117274561,7.04603934030564E7,4.599228369708468,4.653756745508691,0.18308872643487184,2.795387538952672
3,min,Afghanistan,2000,Developed,44.0,1,0,0.01,0.0,2,...,3,0.74,2,0.1,1.68135,34.0,0.1,0.1,0.0,4.2
4,max,Zimbabwe,2015,Developing,89.0,723,1600,17.87,18961.3486,99,...,99,14.39,99,50.6,119172.7418,1.293859294E9,27.2,28.2,0.936,20.7


# Veri On Haziriligi

In [230]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.regression import LinearRegression, LinearRegressionModel



# StringIndexer

In [231]:
status_string_indexer = StringIndexer() \
.setInputCol("Status").setOutputCol("StatusIndexed")

# OneHotEncoder

In [232]:
encoder = OneHotEncoderEstimator() \
.setInputCols(["StatusIndexed"]).setOutputCols(["StatusEncoded"])

# VectorAssembler 

In [233]:
vector_assembler = VectorAssembler() \
.setInputCols(numerical_cols + encoder.getOutputCols()) \
.setOutputCol("features")

In [234]:
numerical_cols + encoder.getOutputCols()

['Year',
 'AdultMortality',
 'InfantDeaths',
 'Alcohol',
 'BMI',
 'UnderFiveDeaths',
 'TotalExpenditure',
 'Diphtheria',
 'HIV_AIDS',
 'GDP',
 'IncomeCompositionOfResources',
 'Schooling',
 'StatusEncoded']

# Linear Regression

In [235]:
linear_regression_object = LinearRegression() \
.setFeaturesCol("features").setLabelCol("label")

# Pipeline

In [236]:
pipeline_object = Pipeline().setStages([status_string_indexer, encoder, vector_assembler, linear_regression_object])

###  buradaki sira onemli listedeki siraya gore islemler gerceklesiyor

# Veri Setini Ayirma

In [237]:
train_df, test_df = df3.randomSplit([0.8, 0.2], seed=142)

# Model Egitimi

In [238]:
train_df.cache()
test_df.cache()

DataFrame[Country: string, Year: int, Status: string, label: double, AdultMortality: int, InfantDeaths: int, Alcohol: double, PercentageExpenditure: double, HepatitisB: int, Measles: int, BMI: double, UnderFiveDeaths: int, Polio: int, TotalExpenditure: double, Diphtheria: int, HIV_AIDS: double, GDP: double, Population: double, Thinness119: double, Thinness59: double, IncomeCompositionOfResources: double, Schooling: double]

In [239]:
pipline_model = pipeline_object.fit(train_df)

In [240]:
pipline_model.transform(test_df).select("label","prediction").toPandas().head()

,label,prediction
0,56.2,63.595831
1,57.3,60.953196
2,58.1,61.949129
3,58.6,62.238897
4,72.6,72.930192


# Linear Modeli Pipeline Model İçersinden Alma

In [241]:
lr_model = pipline_model.stages[-1]

In [242]:
lr_model.coefficients

DenseVector([-0.1306, -0.0167, 0.0884, -0.0933, 0.0335, -0.0673, 0.1014, 0.0144, -0.4472, 0.0001, 9.6162, 0.9347, -0.7855])

In [243]:
lr_model.intercept

315.44452546889534

In [244]:
lr_model.summary.r2

0.8393963848802052

In [245]:
# 0.84 basarisi var

In [246]:
lr_model.summary.rootMeanSquaredError

3.534708903926837

In [247]:
lr_model.summary.pValues

[3.7210678383026163e-07,
 0.0,
 1.554312234475219e-14,
 0.011812369488869878,
 6.888301440355349e-08,
 4.218847493575595e-15,
 0.02289647691655916,
 0.0037352078727632687,
 0.0,
 7.28083149326153e-10,
 0.0,
 0.0,
 0.036279656005715255,
 9.313896320861659e-10]

# Niteliklerle PVALUES leri birlestirme

In [248]:
zipped = zip(lr_model.summary.pValues,numerical_cols+["Status","Sabit"])

In [249]:
sorted_zipped = sorted(zipped)

In [250]:
for i in sorted_zipped:
    print(i)

(0.0, 'AdultMortality')
(0.0, 'HIV_AIDS')
(0.0, 'IncomeCompositionOfResources')
(0.0, 'Schooling')
(4.218847493575595e-15, 'UnderFiveDeaths')
(1.554312234475219e-14, 'InfantDeaths')
(7.28083149326153e-10, 'GDP')
(9.313896320861659e-10, 'Sabit')
(6.888301440355349e-08, 'BMI')
(3.7210678383026163e-07, 'Year')
(0.0037352078727632687, 'Diphtheria')
(0.011812369488869878, 'Alcohol')
(0.02289647691655916, 'TotalExpenditure')
(0.036279656005715255, 'Status')


In [251]:
"""
r2 = 0.8402372243436562
(0.0, 'AdultMortality')
(0.0, 'HIV_AIDS')
(0.0, 'IncomeCompositionOfResources')
(0.0, 'Schooling')
(1.021405182655144e-14, 'UnderFiveDeaths')
(9.037215420448774e-14, 'InfantDeaths')
(8.194222855806288e-09, 'Sabit')
(2.214080842888322e-06, 'Year')
(9.277282233099982e-06, 'BMI')
(0.0069944469886213945, 'Alcohol')
(0.03194558950457549, 'Diphtheria')
(0.033259070668361534, 'Status')
(0.041134342157394865, 'TotalExpenditure')
6.tur(0.14696628529633227, 'Polio')
7. tur(0.20657371462857865, 'Thinness59')
(0.2415689423147689, 'GDP')
5. tur(0.3280541792373892, 'HepatitisB')
4. tur(0.4246739503761421, 'PercentageExpenditure')
3. tur(0.46316882090430034, 'Measles')
2.tur (0.6154505536890638, 'Thinness119')
1. tur (0.9774634776466289, 'Population')
"""

"\nr2 = 0.8402372243436562\n(0.0, 'AdultMortality')\n(0.0, 'HIV_AIDS')\n(0.0, 'IncomeCompositionOfResources')\n(0.0, 'Schooling')\n(1.021405182655144e-14, 'UnderFiveDeaths')\n(9.037215420448774e-14, 'InfantDeaths')\n(8.194222855806288e-09, 'Sabit')\n(2.214080842888322e-06, 'Year')\n(9.277282233099982e-06, 'BMI')\n(0.0069944469886213945, 'Alcohol')\n(0.03194558950457549, 'Diphtheria')\n(0.033259070668361534, 'Status')\n(0.041134342157394865, 'TotalExpenditure')\n6.tur(0.14696628529633227, 'Polio')\n7. tur(0.20657371462857865, 'Thinness59')\n(0.2415689423147689, 'GDP')\n5. tur(0.3280541792373892, 'HepatitisB')\n4. tur(0.4246739503761421, 'PercentageExpenditure')\n3. tur(0.46316882090430034, 'Measles')\n2.tur (0.6154505536890638, 'Thinness119')\n1. tur (0.9774634776466289, 'Population')\n"